# Geração de labirintos com busca em profundidade (DFS)

## Função DFS

In [ ]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import time
import tracemalloc
from collections import deque
import pandas as pd
import sys

import heapq
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm

def dfs_resolver_labirinto_com_visualizacao(labirinto, inicio, fim):
    altura, largura = len(labirinto), len(labirinto[0])
    visitado = [[False for _ in range(largura)] for _ in range(altura)]
    pai = [[None for _ in range(largura)] for _ in range(altura)]

    pilha = deque([])
    nos_visitados_dfs = []

    pilha.append(inicio)

    tracemalloc.start()
    tempo_inicio = time.perf_counter()

    caminho_encontrado = False
    while pilha:
        x, y = pilha.pop()

        if visitado[y][x]:
            continue

        visitado[y][x] = True
        nos_visitados_dfs.append((x,y))

        if (x, y) == fim:
            caminho_encontrado = True
            break

        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < largura and 0 <= ny < altura:
                if labirinto[ny][nx] == 1 and not visitado[ny][nx]:
                    pilha.append((nx, ny))
                    pai[ny][nx] = (x, y)

    tempo_fim = time.perf_counter()
    memoria_usada = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    caminho_final = []
    if caminho_encontrado:
        curr_x, curr_y = fim
        while (curr_x, curr_y) != inicio and pai[curr_y][curr_x] is not None:
            caminho_final.append((curr_x, curr_y))
            curr_x, curr_y = pai[curr_y][curr_x]
        caminho_final.append(inicio)
        caminho_final.reverse()

    return {
        "caminho": caminho_final,
        "nos_visitados": list(nos_visitados_dfs),
        "tempo_ms": (tempo_fim - tempo_inicio) * 1000,
        "memoria_bytes": memoria_usada,
        "comprimento": len(caminho_final) if caminho_encontrado else 0,
        "qtd_nos_visitados": len(nos_visitados_dfs)
    }

## Geração de animações

In [ ]:
def gerar_animacao(labirinto, resultado, caminho_saida, fps=10):
    # Aumentar a altura da figura para acomodar o título
    fig, ax = plt.subplots(figsize=(10, 10.2))  # Altura aumentada em 0.2

    cmap = plt.colormaps['viridis']
    norm = plt.Normalize(vmin=0, vmax=3)

    # Matriz de estado inicial
    estado = np.zeros_like(labirinto)
    estado[labirinto == 0] = 0  # Parede
    estado[labirinto == 1] = 1  # Livre

    img = ax.imshow(estado, cmap=cmap, norm=norm, interpolation='nearest')
    ax.axis('off')

    # Ajustes críticos para evitar corte
    plt.subplots_adjust(top=0.95)  # Reserva 5% do topo para o título
    fig.tight_layout(rect=[0, 0, 1, 0.95])  # Define área útil da figura

    writer = FFMpegWriter(fps=fps)
    total_nos = len(resultado['nos_visitados'])
    step = max(1, total_nos // 100)  # Máximo de 100 frames para exploração

    with writer.saving(fig, caminho_saida, dpi=100):
        # Frame inicial
        writer.grab_frame()

        # Fase de exploração
        for i in tqdm(range(0, total_nos, step), desc="Gerando frames"):
            bloco = resultado['nos_visitados'][i:i+step]
            for x, y in bloco:
                if estado[y][x] == 1:  # Só atualiza se for espaço livre
                    estado[y][x] = 3  # Visitado
            img.set_array(estado)
            ax.set_title(f"Exploração: {min(i+step, total_nos)}/{total_nos} nós", fontsize=10)  # Fonte reduzida
            writer.grab_frame()

        # Fase do caminho (se existir)
        if resultado['caminho']:
            for x, y in resultado['caminho']:
                estado[y][x] = 2  # Caminho
            img.set_array(estado)
            ax.set_title(f"Caminho encontrado! Comprimento: {len(resultado['caminho'])}", fontsize=10)
            for _ in range(30):  # 3 segundos de exibição final
                writer.grab_frame()
        else:
            ax.set_title("Caminho não encontrado!", fontsize=10)
            for _ in range(30):
                writer.grab_frame()

    plt.close(fig)

## Geração de imagens estáticas

In [ ]:
# Função para gerar imagens estáticas
def gerar_imagem(labirinto, resultado, caminho_saida, titulo):
    visualizacao = np.zeros_like(labirinto)
    visualizacao[labirinto == 0] = 0  # Parede
    visualizacao[labirinto == 1] = 1  # Livre

    # Marcar nós visitados (exceto caminho)
    for x, y in resultado['nos_visitados']:
        if visualizacao[y][x] == 1:
            visualizacao[y][x] = 3  # Visitado

    # Marcar caminho
    for x, y in resultado['caminho']:
        visualizacao[y][x] = 2  # Caminho

    plt.figure(figsize=(10, 10))
    plt.imshow(visualizacao, cmap=plt.colormaps['viridis'], vmin=0, vmax=3)
    plt.axis('off')
    plt.title(titulo)

    # Criar legenda
    legenda = [
        mpatches.Patch(color=plt.colormaps['viridis'](0/3), label='Parede'),
        mpatches.Patch(color=plt.colormaps['viridis'](1/3), label='Espaço livre'),
        mpatches.Patch(color=plt.colormaps['viridis'](2/3), label='Caminho final'),
        mpatches.Patch(color=plt.colormaps['viridis'](3/3), label='Nós visitados')
    ]
    plt.legend(handles=legenda, loc='upper right', bbox_to_anchor=(1.15, 1.0))

    plt.savefig(caminho_saida, bbox_inches='tight')
    plt.close()

## Configuração de pastas

In [ ]:
# Configurações principais
pastas = {
    "pequenos": "pequeno",
    "medios": "medio",
    "medio-grande": "medio-grande",
    "grandes": "grande"
}
abordagem = "dfs"

# Criar estrutura de diretórios
os.makedirs("midias", exist_ok=True)
for categoria in ["imagens", "animacoes"]:
    for tamanho in pastas.keys():
        os.makedirs(f"midias/{categoria}/{abordagem}/{tamanho}", exist_ok=True)


## Processamento dos labirintos

In [ ]:
# Processar todos os labirintos
for tamanho_pasta, tamanho_nome in pastas.items():
    print(f"\nProcessando labirintos {tamanho_nome}...")

    for i in range(1, 21):
        nome_arquivo = f"labirinto_{i:02d}.npy"
        lab_path = os.path.join("labirintos", tamanho_pasta, nome_arquivo)
        print(f"\nProcessando: {nome_arquivo}")

        try:
            # Carregar labirinto
            labirinto = np.load(lab_path)
            entrada = (0, 1)
            saida = (labirinto.shape[1]-1, labirinto.shape[0]-2)

            # Executar algoritmo
            resultado = dfs_resolver_labirinto_com_visualizacao(labirinto, entrada, saida)
            sucesso = bool(resultado['caminho'])

            # Gerar imagem estática
            img_path = f"midias/imagens/{abordagem}/{tamanho_pasta}/{nome_arquivo.replace('.npy', '.png')}"
            gerar_imagem(
                labirinto,
                resultado,
                img_path,
                f"Labirinto {tamanho_nome} - {nome_arquivo}"
            )
            print(f"Imagem gerada: {img_path}")

            # Gerar animação (apenas se encontrou caminho)
            if sucesso:
                anim_path = f"midias/animacoes/{abordagem}/{tamanho_pasta}/{nome_arquivo.replace('.npy', '.mp4')}"
                gerar_animacao(labirinto, resultado, anim_path)
                print(f"Animação gerada: {anim_path}")
            else:
                print("Caminho não encontrado. Animação não gerada.")

        except Exception as e:
            print(f"Erro no processamento: {str(e)}")

print("\nProcessamento concluído com sucesso!")


Processando labirintos pequeno...

Processando: labirinto_01.npy
Imagem gerada: midias/imagens/dfs/pequenos/labirinto_01.png


Gerando frames: 100%|██████████| 137/137 [00:19<00:00,  7.15it/s]


Animação gerada: midias/animacoes/dfs/pequenos/labirinto_01.mp4

Processando: labirinto_02.npy
Imagem gerada: midias/imagens/dfs/pequenos/labirinto_02.png


Gerando frames: 100%|██████████| 133/133 [00:17<00:00,  7.48it/s]


Animação gerada: midias/animacoes/dfs/pequenos/labirinto_02.mp4

Processando: labirinto_03.npy
Imagem gerada: midias/imagens/dfs/pequenos/labirinto_03.png


...

Animação gerada: midias/animacoes/dfs/medio-grande/labirinto_07.mp4

Processando: labirinto_08.npy
Imagem gerada: midias/imagens/dfs/medio-grande/labirinto_08.png


Gerando frames: 100%|██████████| 101/101 [00:19<00:00,  5.16it/s]


Animação gerada: midias/animacoes/dfs/medio-grande/labirinto_08.mp4

Processando: labirinto_09.npy
Imagem gerada: midias/imagens/dfs/medio-grande/labirinto_09.png


Gerando frames: 100%|██████████| 101/101 [00:20<00:00,  5.05it/s]


Animação gerada: midias/animacoes/dfs/medio-grande/labirinto_09.mp4

Processando: labirinto_10.npy


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("midias_dfs", "zip", "/content/midias")
files.download("midias_dfs.zip")